In [1]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import lazyqsar
import pickle
import h5py
import tqdm
import sys
import os

# alpha = int(sys.argv[1])

# Define root directory
root = "."

# Load pickle
# pathogen_code, file, column = pickle.load(open(os.path.join(root, "..", "..", "tmp", "models_to_train.pkl"), "rb"))[alpha]
pathogen_code = 'mtuberculosis'
file = "CHEMBL4649971_PERCENTEFFECT_%"
file = "CHEMBL4649971_IC50_umol.L-1"
file = "CHEMBL1794426_EC50_umol.L-1"
file = "CHEMBL4649947_PERCENTEFFECT_%"

In [2]:
# Load data
sys.stderr.write(str(pathogen_code) + "\n" + str(file))
sys.stderr.write("\n\n")
sys.stderr.flush()
data = pd.read_csv(os.path.join(root, "..", "output", pathogen_code, "datasets", f"{file}.csv.gz"))
X = data['canonical_smiles'].astype(str).tolist()
Y = data['bin'].astype(np.int8).to_numpy()

# Define stratified 5 fold CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Load descriptors
sys.stderr.write("Loading descriptors...")
sys.stderr.write("\n\n")
sys.stderr.flush()
with h5py.File(os.path.join(root, "..", "output", pathogen_code, "descriptors.h5"), "r") as f:
    SMILES = f['SMILES'][:]
    X_morgan = f['X_morgan'][:]
    ### add here rdkit descriptors

# Define dict mapping smiles to morgan fingerprints
SMILES_TO_MORGAN = {
    smiles.decode("utf-8"): fp
    for (chembl_id, smiles), fp in zip(SMILES, X_morgan)}

mtuberculosis
CHEMBL4649947_PERCENTEFFECT_%

Loading descriptors...



In [3]:
from lazyqsar.agnostic import LazyBinaryClassifier

# For each split
AUROCS = []
for c, (train, test) in enumerate(skf.split(X, Y)):
    
    sys.stderr.write(f"Training split {c}...")
    sys.stderr.write("\n")
    sys.stderr.flush()

    # Get train/test indices
    X_train, Y_train = [X[i] for i in train], Y[train]
    X_test, Y_test = [X[i] for i in test], Y[test]

    # Get Morgan Fingerprints and train model
    X_train = np.array([SMILES_TO_MORGAN[smi] for smi in X_train])
    X_test  = np.array([SMILES_TO_MORGAN[smi] for smi in X_test])
    model = LazyBinaryClassifier(mode="fast")
    model.fit(X=X_train, y=Y_train)

    break



/home/acomajuncosa/miniconda3/envs/lazyqsar/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training split 0...


12:21:52 INFO     Dataset summary:                                                                                 
                    • Total samples: 7072                                                                          
                    • Positive samples: 9                                                                          
                    • Negative samples: 7063                                                                       
                    • Max samples per partition: 100000                                                            
                    • Min samples per partition: 30                                                                
                    • Positive proportion: 0.00

         INFO     Original positive samples: 9, total samples: 7072

         INFO     Maximum samples: 100000

         DEBUG    Sampling 9 positive and 7063 negative samples from 7072 total samples.

...

12:38:50 INFO     Fitting completed

         INFO     Individual model scores:

         INFO     Model weights: [0.04342673 0.14280557 0.05243307 0.07554086 0.15297749 0.02503851                
                   0.13475872 0.05615021 0.15841839 0.15845045]

         INFO     Model has successfull been fitted!

         INFO     Fitting completed in 1018.20 seconds.

In [4]:
#     # Store results
#     auroc = round(roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1]), 3)
#     AUROCS.append(auroc)
#     sys.stderr.write(f"AUROC: {auroc}")
#     sys.stderr.write("\n\n")
#     sys.stderr.flush()

# # Train full model
# X = np.array([SMILES_TO_MORGAN[smi] for smi in X])
# model = LazyBinaryClassifier(mode="default")
# model.fit(X=X, y=Y)

# # Save model
# model.save(os.path.join(root, "..", "..", "output", pathogen_code, "models", file.replace(".csv.gz", ".zip")))

# # Save CV results
# with open(os.path.join(root, "..", "..", "output", pathogen_code, "models", file.replace('.gz', '')), "w") as outfile:
#     outfile.write(",".join([str(i) for i in AUROCS]))